In [1]:
### Create a Spark Object
from pyspark.sql import SparkSession

#spark = SparkSession \
#        .builder \
#        .master('yarn') \
#        .appName("Python Spark SQL basic example") \
#        .getOrCreate()

spark = SparkSession \
        .builder \
        .master("local[*]") \
        .appName("Python Spark SQL basic example") \
        .getOrCreate()

print("Spark Object id created ...")
spark.stop()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/01/26 08:53:39 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/01/26 08:53:45 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


Spark Object id created ...
